Agents List:

- Fully static recommendation Agent
- Simple random recommendation Agent
- Logistic regression Agent
- Greedy bandit Agent
- Greedy organic Agent
- CF SVD Agent
- Organic MF agent
- Bandit MF Agent
- Bandit Regression Agent
- Organic user count Agent
- Neural network Ips Agent

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
from numpy.random.mtrand import RandomState
from recogym import Configuration
from recogym.agents import Agent
from scipy.special import logsumexp
import pandas as pd
from scipy.stats.distributions import beta
from copy import deepcopy
from recogym.agents import OrganicUserEventCounterAgent, organic_user_count_args

from recogym.envs.observation import Observation
from recogym.agents import RandomAgent, random_args
from recogym import verify_agents, verify_agents_IPS
from recogym.evaluate_agent import plot_verify_agents, verify_agents_recall_at_k
from tensorflow.keras.utils import to_categorical

import gym
import numpy as np
from recogym import env_1_args, Configuration

# Set style for pretty plots
plt.style.use('ggplot')

P = 10 # Number of Products
U = 10 # Number of Users

# You can overwrite environment arguments here:
env_1_args['random_seed'] = 42
env_1_args['num_products']= P
env_1_args['phi_var']=0.0
env_1_args['number_of_flips']=8
env_1_args['sigma_mu_organic'] = 0.0
env_1_args['sigma_omega']=0
#env_1_args['normalize_beta']=True
# Initialize the gym for the first time by calling .make() and .init_gym()
env = gym.make('reco-gym-v1')
env.init_gym(env_1_args)

env.reset()

ImportError: Traceback (most recent call last):
  File "/usr/local/lib/python3.8/site-packages/tensorflow/python/pywrap_tensorflow.py", line 58, in <module>
    from tensorflow.python.pywrap_tensorflow_internal import *
  File "/usr/local/lib/python3.8/site-packages/tensorflow/python/pywrap_tensorflow_internal.py", line 28, in <module>
    _pywrap_tensorflow_internal = swig_import_helper()
  File "/usr/local/lib/python3.8/site-packages/tensorflow/python/pywrap_tensorflow_internal.py", line 24, in swig_import_helper
    _mod = imp.load_module('_pywrap_tensorflow_internal', fp, pathname, description)
  File "/usr/local/Cellar/python@3.8/3.8.2/Frameworks/Python.framework/Versions/3.8/lib/python3.8/imp.py", line 242, in load_module
    return load_dynamic(name, filename, file)
  File "/usr/local/Cellar/python@3.8/3.8.2/Frameworks/Python.framework/Versions/3.8/lib/python3.8/imp.py", line 342, in load_dynamic
    return _load(spec)
ImportError: dlopen(/usr/local/lib/python3.8/site-packages/t

In [ ]:
# Generate RecSys logs for U users
reco_log = env.generate_logs(U)
reco_log.head(20)

In [ ]:
from collections import defaultdict
from sklearn.linear_model import LogisticRegression

def create_embeddings(reco_log, U, P, K = 5):
    organic_reco_log = reco_log[reco_log['z']=='organic']
    counts = np.zeros((U,P))

    for u in range(U):
        counts[u,:] = to_categorical(np.array(organic_reco_log[organic_reco_log['u']==u].v,dtype=int),P).sum(0)

    counts_above_zero = 1.*(counts>0) # above zero counts only
    co_counts = np.matmul(counts_above_zero.T,counts_above_zero)

    w , v = np.linalg.eig(co_counts)
    idx = np.argsort(w)[::-1]
    v = np.real(v[:,idx])
    w = np.real(w[idx])

    wdash = np.zeros_like(w)

    wdash[0:K] = w[0:K]
    embeddings = np.matmul(v,np.sqrt(np.diag(wdash)))[:,0:K] # keep the non-zero components
    return embeddings

In [ ]:
organic_counter_agent = OrganicUserEventCounterAgent(Configuration({
            **organic_user_count_args,
            **env_1_args,
            'select_randomly': True,
        }))

random_args['num_products'] = P

agent_rand = RandomAgent(Configuration({**env_1_args, **random_args,}))

## A/B-Test Evaluation

In [ ]:
result_AB = verify_agents(env, U, {' Random': agent_rand, 'Popularity': organic_counter_agent})
result_AB

In [ ]:
fig = plot_verify_agents(result_AB)
plt.ylabel('CTR')
plt.show()

